#### INST326 OOP Project 04

Rename this notebook, replacing "_Assignment" with "_YourName"<br>
Insert Signature Block Here

#### Christian Sorensen
> INST326
> Project 04
> 11/25/2024
#### Honor Pledge
> I pledge that the work contained in this assignment is my own, and that I have complied with University and course policies on academic integrity and AI use.


You may work as an individual on **ONE** of the following projects, **OR** if you want to work as a group, contact Dr. Dempwolf for a project assignment. That group assignment will be part of an ongoing research project analyzing innovation ecosystems. 

### Individual Projects
Choose **ONE** of the following projects and write to code solution in the code cell below your choice. Use comments in your code to document your solution. If you need to write comments to the grader, add a markdown cell immediately above your code solution and add your comments there. Be sure to read and follow the Notebook Instructions at the bottom of this notebook. Your grade may depend on it! 

#### 1. Library Management System
>  Objective: Develop a system to manage a library’s collection of books, users, and loan records. This system should allow users to borrow and return books, as well as track which books are currently available.
>
> Requirements
>>- Use classes to represent books, users, and the library.
>>- Implement encapsulation to protect class attributes.
>>- Use inheritance to handle different types of users (e.g., students and teachers).
>>- Demonstrate polymorphism in borrowing rules (e.g., different borrowing limits for students vs. teachers).
>>- Include methods for adding/removing books, registering users, and managing book loans.
>>- Include execution code to demonstrate that your solution works

In [ ]:
from __future__ import annotations
import csv
from abc import ABC



class Book:
    """Book class, with attributes for title, author, and year of publication"""
    def __init__(self, title: str, author: str = "Unknown", year: str = "Unknown"):
        self.title = title
        self.author = author
        self.year = year

    def __repr__(self):
        return self.title
    
    def __eq__(self, book_2):
        """Compare if two books are the same"""
        if not isinstance(book_2, Book):
            raise TypeError("Cannot compare Book object with non-Book object")
        else:
            for attr in self.__dict__: # Checks if all the shared attributes between the books are identical, so that comparisons with LibraryBook objects also work
                if attr not in book_2.__dict__:
                    continue
                if getattr(self, attr) != getattr(book_2, attr):
                    return False # If the attributes do not match, return False
            return True
            
    
class LibraryBook(Book):
    """LibraryBook objects are used internally as 'templates', and hold information about a book in the library catalogue.
    Physical copies of books are tracked using unique sub-ids objects, since attributes of the LibraryBook itself are likely to be identical between them"""
    def __init__(self,  book_id: int, title: str, author: str = "Unknown", year: str = "Unknown"):
        super().__init__(title, author, year)
        self.book_id = book_id # Unique identifier for book, for differentiating different versions of the same book, or books with the same title
        self.copies = 1 # Number of copies of this book in the catalogue

    @property
    def book_id(self) -> str:
        return self.__book_id
    
    @book_id.setter
    def book_id(self, new_id: int):
        if type(new_id) is int and new_id >= 0: # Ensure new id is a non-negative integer
            new_id = f"{new_id:3f}"
            self.__book_id = new_id
        else:
            raise ValueError("Id must be non-negative integer")


class User(ABC):
    """Base User abstract class. Subclasses can implement restrictions on loaning books"""
    def __init__(self, user_id: int, firstname: str, lastname: str):
        self.user_id = user_id # Unique identification number to differentiate between users with identical names
        self.fname = firstname
        self.lname = lastname

    def __repr__(self):
        """Object is represented by lastname, firstname"""
        return f"{self.lname}, {self.fname}"

    @property
    def user_id(self) -> str:
        return self.__user_id
    
    @user_id.setter
    def user_id(self, new_id: int):
        """Ensure new id is a non-negative integer, and stores it as a formatted string"""
        if type(new_id) is int and new_id >= 0:
            new_id = f"{new_id}"
            self.__user_id = new_id
        else:
            raise ValueError("Id must be non-negative integer")


class Student(User):
    """Student class. Has a loan limit of 10"""
    def __init__(self, user_id: int, firstname: str, lastname: str):
        super().__init__(user_id, firstname, lastname)
        self.loan_limit = 10


class Teacher(User):
    """Teacher class. Has a loan limit of 20"""
    def __init__(self, user_id: int, firstname: str, lastname: str):
        super().__init__(user_id, firstname, lastname)
        self.loan_limit = 10



class LibraryManager:
    def __init__(self):
        self.__users = {}
        self.__catalogue = {}
        self.__loans = {}

    def add_student(self, first_name: str, last_name: str):
        """Adds a new student to the system"""
        user_id = 0
        while str(user_id) in self.__users:
            user_id += 1
        student = Student(user_id, first_name, last_name)
        self.__users.update({str(user_id):student}) # Adds student to the users dict, with their user_id as the key

    def add_teacher(self, first_name: str, last_name: str):
        """Adds a new teacher to the system"""
        user_id = 0
        while str(user_id) in self.__users: # Gets the lowest available id
            user_id += 1
        teacher = Teacher(user_id, first_name, last_name)
        self.__users.update({str(user_id):teacher}) # Adds teacher to the users dict, with their user_id as the key

    def add_library_book(self, book: Book):
        """Adds a new book to the catalogue and make a copy available for loans. If the book already exists in the catalogue, it just makes a new copy"""
        for index, (book_id, library_book) in enumerate(self.__catalogue.items()): # Checks if the book has already been registered in the catalogue before
            if book == library_book:
                self.__add_copy(library_book) # Adds a new copy of the book
                return
        book_id = 0 # If the book is not already in the catalogue, generates a new id for it
        while str(book_id) in self.__catalogue: # Gets the lowest available book_id
            book_id += 1
        library_book = LibraryBook(book_id, book.title, book.author, book.year)
        book_id = library_book.book_id
        if book_id not in self.__catalogue:
            self.__catalogue.update({book_id:library_book}) # Adds the book to the catalogue
            self.__loans.update({book_id:{"0":None}}) # Adds the book to the loan pool
            print(f"Added '{library_book.title}' to catalogue")

    def __add_copy(self, book: LibraryBook) -> str:
        """Adds a copy of the book to the loan pool. Returns the sub_id of the new copy"""
        book_id = book.book_id
        sub_id = 0
        while str(sub_id) in self.__loans[book_id]: # Gets the lowest available sub_id
            sub_id += 1
        self.__loans[book_id].update({str(sub_id):None})
        book = self.get_book_from_id(book_id)
        book.copies += 1
        print(f"Added new copy of '{book.title}' to loan pool")
        return str(sub_id)
    
    def get_book_from_id(self, book_id: str) -> LibraryBook:
        """Returns the LibraryBook object from a given id"""
        if str(book_id) in self.__catalogue:
            return self.__catalogue[str(book_id)]
        else:
            print("book_id does not exist in catalogue")

    def get_book_id(self, book: Book) -> str | None:
        """Gets the LibraryBook id for a given book, or None if the book is not in the catalogue"""

    def is_book_available(self, book: Book) -> str | bool:
        """Checks if a book has an available copy. Returns the sub_id of the available copy, or False if unavailable"""
        for _, (book_id, library_book) in enumerate(self.__catalogue.items()): # Checks each book in the catalogue
            if book == library_book: # Checks if the Book object matches the current LibraryBook object
                loan_dict = self.__loans[book_id]
                for _, (sub_id, loaned_to) in enumerate(loan_dict.items()): # Checks each sub_id to see if an available copy exists
                    if loaned_to == None:
                        return sub_id
                return False
        print("Book not in catalogue")
        return False



if __name__ == "__main__":
    library = LibraryManager()
    book1 = Book("To Kill A Mockingbird", "Harper Lee", "1960")
    print(library.is_book_available(book1))
    library.add_library_book(book1)
    print(library.is_book_available(book1))

    library.add_library_book(book1)
    


Book not in catalogue
False
Added 'To Kill A Mockingbird' to catalogue
0
Added new copy of 'To Kill A Mockingbird' to loan pool


#### 2. Online Shopping Cart System
>  Objective: Build a shopping cart system for an online store that manages products, shopping carts, and orders.
>
> Requirements
>>- Use classes to represent products, shopping carts, and orders.
>>- Implement encapsulation to handle product prices and cart contents securely.
>>- Use inheritance to create different types of products (e.g., electronics, clothing).
>>- Demonstrate polymorphism by calculating discounts based on product type.
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below


#### 3. Restaurant Reservation System
>  Objective: Create a reservation system for a restaurant that manages tables, reservations, and customers.
>
>  Requirements
>>- Use classes to represent tables, customers, and reservations.
>>- Implement encapsulation for managing table availability and reservation details.
>>- Use inheritance to differentiate between walk-in and advance reservations.
>>- Demonstrate polymorphism by handling special cases (e.g., priority seating for VIP customers).
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below


#### 4. Vehicle Rental System
>  Objective: Develop a vehicle rental system that manages a fleet of vehicles, customer rentals, and payment processing.
>
>  Requirements
>>- Use classes to represent different types of vehicles, customers, and rental transactions.
>>- Implement encapsulation to handle sensitive information like customer payment details.
>>- Use inheritance to differentiate between various vehicle types (e.g., cars, trucks, motorcycles).
>>- Demonstrate polymorphism by applying different rental pricing strategies based on vehicle type.
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below


#### 5. Online Learning Platform
>  Objective: Create an online learning platform that manages courses, students, and instructors.
>  
>  Requirements
>>- Use classes to represent courses, students, and instructors.
>>- Implement encapsulation to manage sensitive information like student grades.
>>- Use inheritance to handle different types of courses (e.g., free, paid, and premium).
>>- Demonstrate polymorphism in applying different grading schemes for assignments.
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below


#### 6. E-Commerce Order Processing System
>  Objective: Build an order processing system for an online store that manages products, customers, and orders.
>  
>  Requirements
>>- Use classes to represent products, customers, and orders.
>>- Implement encapsulation for handling payment details securely.
>>- Use inheritance for different types of products (e.g., physical goods, digital downloads).
>>- Demonstrate polymorphism by applying different shipping costs based on product type.
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below


### Notebook Instructions
> Before turning in your notebook:
> 1. Make sure you have renamed the notebook file as instructed
> 2. Make sure you have included your signature block and that it is correct according to the instructions
> 3. comment your code as necessary
> 4. run all code cells and double check that they run correctly. Include you execution code in your submission. If you can't get your code to run correctly and you want partial credit, add a note for the grader in a new markdown cell directly above your code solution.<br><br>
Turn in your notebook by uploading it to ELMS<br>
IF the exercises involve saved data files, put your notebook and the data file(s) in a zip folder and upload the zip folder to ELMS